<a href="https://colab.research.google.com/github/zalgorythm/Evolite/blob/main/now_everything_again.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# api_config.py
API_KEY = "b5966e3a-3e2b-49d4-a60d-3ce716b5ef7c"
API_SECRET = "VBEyElDzjovK4CPdlEYiTylesa2uRl3Yof6bEPc-Y0Q2ODhkN2U1Mi05YTVlLTQwYTYtODMwYS1hODlmMDdiYTIxYmI"

In [10]:
from flask import Flask, jsonify
from flask_cors import CORS
import api_config  # Import the api_config.py file

app = Flask(__name__)
CORS(app) # Enable CORS for cross-origin requests from the HTML

@app.route('/api/keys')
def get_api_keys():
    return jsonify({'apiKey': api_config.API_KEY, 'apiSecret': api_config.API_SECRET})

if __name__ == '__main__':
    app.run(debug=True, port=5000) # Run Flask app on port 5000

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
!pip install Flask-Cors # Install the flask_cors package

```html
<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>16-Bit Coin Terminal</title>
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
  <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/axios/dist/axios.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/crypto-js/4.1.1/crypto-js.min.js"></script>
  <style>
    /* CSS remains the same */
    body {
      background-color: #000;
      font-family: monospace;
      color: #0f0;
    }

    .card {
      background-color: #222;
      border: 2px solid #0f0;
    }

    .card-title {
      color: #0f0;
      font-size: 1.2rem;
    }

    .btn-success {
      background-color: #0f0;
      color: #000;
      border: none;
    }

    .btn-danger {
      background-color: #f00;
      color: #000;
      border: none;
    }

    .form-control {
      background-color: #000;
      color: #0f0;
      border: 1px solid #0f0;
    }

    body::before {
      content: "";
      position: fixed;
      top: 0;
      left: 0;
      width: 100%;
      height: 100%;
      background-image: repeating-linear-gradient(transparent, transparent 1px, rgba(0, 0, 0, 0.1) 1px, rgba(0, 0, 0, 0.1) 2px);
      pointer-events: none;
      z-index: 999;
    }

    .pixel-text {
      font-size: 16px;
      image-rendering: pixelated;
      -webkit-font-smoothing: none;
      -moz-osx-font-smoothing: grayscale;
    }

    .order-book,
    .order-history,
    .positions,
    #orderBookDisplay {
      background-color: #000;
      border: 1px solid #0f0;
      overflow-y: auto;
      max-height: 200px;
    }


    .order-item,
    .position-item,
    .orderbook-item {
      background-color: #222;
      border-bottom: 1px solid #0f0;
      color: #0f0;
      cursor: pointer;
      display: flex;
      justify-content: space-between;
      align-items: center;
      padding: 0.5rem;
    }

    .order-actions {
      display: flex;
      gap: 5px;
    }


    .loading-overlay {
      position: fixed;
      top: 0;
      left: 0;
      width: 100%;
      height: 100%;
      background-color: rgba(0, 0, 0, 0.5);
      display: flex;
      justify-content: center;
      align-items: center;
      z-index: 1000;
      display: none;
    }

    .loading-spinner {
      border: 4px solid #f3f3f3;
      border-top: 4px solid #3498db;
      border-radius: 50%;
      width: 50px;
      height: 50px;
      animation: spin 2s linear infinite;
    }

    @keyframes spin {
      0% {
        transform: rotate(0deg);
      }

      100% {
        transform: rotate(360deg);
      }
    }

    .api-prompt {
      position: fixed;
      top: 50%;
      left: 50%;
      transform: translate(-50%, -50%);
      background-color: #222;
      color: #00ff9d;
      padding: 20px;
      border-radius: 5px;
      z-index: 1001;
    }

    .api-prompt input {
      background-color: #333;
      color: #00ff9d;
      border: none;
      padding: 5px;
      margin-bottom: 10px;
      width: 100%;
    }

    .api-prompt button {
      background-color: #00ff9d;
      color: #222;
      border: none;
      padding: 8px 15px;
      border-radius: 5px;
      cursor: pointer;
    }

    .terminal-container {
      background: #1a1a1a;
      color: #00ff9d;
      min-height: 100vh;
      padding: 20px;
    }

    .price-chart {
      height: 400px;
    }

    .order-book {
      height: 300px;
      overflow-y: auto;
    }

    .order-history {
      height: 300px;
      overflow-y: auto;
    }

    .positions {
      height: 200px;
      overflow-y: auto;
    }

    #orderBookDisplay {
      height: 200px;
    }


    .error-message {
      color: red;
      margin-top: 10px;
    }

    /* Style for the modify order modal */
    .modify-order-modal {
      display: none;
      position: fixed;
      z-index: 1002;
      left: 0;
      top: 0;
      width: 100%;
      height: 100%;
      overflow: auto;
      background-color: rgba(0, 0, 0, 0.8);
    }

    .modify-order-modal-content {
      background-color: #222;
      margin: 15% auto;
      padding: 20px;
      border: 1px solid #0f0;
      width: 80%;
      color: #0f0;
      position: relative;
    }

    .modify-order-modal-content .close-button {
      color: #0f0;
      position: absolute;
      top: 0;
      right: 0;
      padding: 10px;
      cursor: pointer;
      font-size: 1.5em;
    }

    .modify-order-modal-content label {
      display: block;
      margin-bottom: 8px;
    }

    .modify-order-modal-content input[type="number"] {
      width: 100%;
      padding: 8px;
      margin-bottom: 15px;
      background-color: #000;
      color: #0f0;
      border: 1px solid #0f0;
    }

    .modify-order-modal-content button {
      background-color: #0f0;
      color: #000;
      border: none;
      padding: 10px 20px;
      border-radius: 5px;
      cursor: pointer;
    }
  </style>
</head>

<body class="terminal-container">
  <div class="loading-overlay" id="loadingOverlay">
    <div class="loading-spinner"></div>
  </div>

  <div class="api-prompt" id="apiPrompt" style="display:none;">
    <h3 class="pixel-text">Enter API Keys</h3>
    <input type="text" id="apiKeyInput" placeholder="API Key">
    <input type="password" id="apiSecretInput" placeholder="API Secret">
    <button id="saveApiKeys">Save</button>
    <div id="apiError" class="error-message"></div>
  </div>

  <div id="modifyOrderModal" class="modify-order-modal">
    <div class="modify-order-modal-content">
      <span class="close-button" id="closeModifyModal">&times;</span>
      <h3 class="pixel-text">Modify Order</h3>
      <div id="modifyOrderError" class="error-message"></div>
      <label for="modifyQuantity">New Quantity</label>
      <input type="number" id="modifyQuantity" class="form-control bg-dark text-success" placeholder="New Quantity">
      <button id="confirmModifyOrder" class="btn btn-success">Confirm Modify</button>
    </div>
  </div>


  <div class="container-fluid">
    <div class="row">
      <div class="col-md-8">
        <div class="card bg-dark">
          <div class="card-body">
            <canvas id="priceChart"></canvas>
          </div>
        </div>
      </div>
      <div class="col-md-4">
        <div class="card bg-dark mb-3">
          <div class="card-body">
            <h5 class="card-title pixel-text">Order Book</h5>
            <div id="orderBookDisplay" class="order-book"></div>
          </div>
        </div>
        <div class="card bg-dark mb-3">
          <div class="card-body">
            <h5 class="card-title pixel-text">Quick Trade</h5>
            <div id="priceTicker" class="mb-3 fs-4 text-center">Loading...</div>
            <div class="mb-3">
              <input type="number" id="quantity" class="form-control bg-dark text-success" placeholder="Quantity">
            </div>
            <div class="d-grid gap-2">
              <button class="btn btn-success" id="buyButton">Buy</button>
              <button class="btn btn-danger" id="sellButton">Sell</button>
            </div>
            <div id="orderError" class="error-message"></div>
          </div>
        </div>
        <div class="card bg-dark mb-3">
          <div class="card-body">
            <h5 class="card-title pixel-text">Trading Strategy</h5>
            <div class="mb-3">
              <select class="form-control bg-dark text-success" id="strategySelector">
                <option value="none">None (Manual Trading)</option>
                <option value="ma_crossover">Moving Average Crossover (Example)</option>
              </select>
            </div>
            <div id="strategyStatus" class="text-center">Strategy: Inactive</div>
            <div id="strategySignal" class="text-center">Signal: NONE</div> </div>
        </div>
        <div class="card bg-dark mb-3">
          <div class="card-body">
            <h5 class="card-title pixel-text">Open Orders</h5>
            <div id="openOrders" class="order-book"></div>
          </div>
        </div>
        <div class="card bg-dark mb-3">
          <div class="card-body">
            <h5 class="card-title pixel-text">Order History</h5>
            <div id="orderHistory" class="order-history"></div>
          </div>
        </div>
        <div class="card bg-dark">
          <div class="card-body">
            <h5 class="card-title pixel-text">Positions</h5>
            <div id="positions" class="positions"></div>
          </div>
        </div>
        <div class="card bg-dark">
          <div class="card-body">
            <h5 class="card-title pixel-text">Trading Risks</h5>
            <p class="pixel-text text-warning small">
              Crypto trading is risky. AI strategies are not foolproof. Market volatility, security risks, and technical issues can lead to losses. Use at your own risk.
            </p>
          </div>
        </div>
      </div>
    </div>
  </div>

  <script>
    const BASE_URL = 'https://api.phemex.com';
    const SYMBOL = 'BTCUSD';
    let API_KEY = null; // API Keys will be fetched dynamically
    let API_SECRET = null;
    let priceChart;
    let currentPrice = 0;
    let ws;

    // ------ Utility Functions Module ------
    const utils = {
      showLoadingOverlay: () => document.getElementById('loadingOverlay').style.display = 'flex',
      hideLoadingOverlay: () => document.getElementById('loadingOverlay').style.display = 'none',
      displayError: (message, isCritical = true) => {
        console.error('Terminal Error:', message);
        if (isCritical) {
          document.getElementById('orderError').textContent = message;
        } else {
          console.warn('Non-critical error:', message);
          alert(`Non-critical error: ${message}`); // Consider toast notifications for future
        }
      }
    };


    // ------ API Request Module ------
    const apiRequest = {
      signedRequest: async (endpoint, method, data = {}) => {
        const apiKey = API_KEY; // Use dynamically fetched API_KEY
        const apiSecret = API_SECRET; // Use dynamically fetched API_SECRET

        if (!apiKey || !apiSecret) {
          throw new Error("API Keys not loaded. Please ensure Flask server is running and serving keys.");
        }

        const expires = Math.floor(Date.now() / 1000) + 60;
        const signature = CryptoJS.HmacSHA256(method + endpoint + expires + JSON.stringify(data), apiSecret).toString();

        const headers = {
          'Content-Type': 'application/json',
          'X-Phemex-API-Key': apiKey,
          'X-Phemex-Request-Expiry': expires,
          'X-Phemex-Request-Signature': signature
        };

        const config = {
          method: method,
          url: `${BASE_URL}/api/v1${endpoint}`,
          headers: headers,
          data: data
        };

        try {
          const response = await axios(config);
          return response.data;
        } catch (error) {
          console.error('API request error:', error);
          if (error.message === "API Keys not loaded. Please ensure Flask server is running and serving keys.") {
            utils.displayError(error.message);
          } else {
            const errorMessage = error.response?.data?.error?.message || "API request failed. Check console for details.";
            utils.displayError(errorMessage, false);
          }
          throw error;
        }
      }
    };

    // ------ WebSocket Module ------
    const websocketModule = {
      ws: null,
      initWebSocket: () => {
        websocketModule.ws = new WebSocket('wss://ws.phemex.com');

        websocketModule.ws.onopen = () => {
          websocketModule.ws.send(JSON.stringify({
            id: 1,
            method: 'trade.subscribe',
            params: [SYMBOL]
          }));
          websocketModule.ws.send(JSON.stringify({
            id: 2,
            method: 'order.subscribe',
            params: [SYMBOL]
          }));
          websocketModule.ws.send(JSON.stringify({
            id: 3,
            method: 'orderbook.subscribe',
            params: [SYMBOL]
          }));
        };

        websocketModule.ws.onmessage = (event) => {
          const data = JSON.parse(event.data);
          if (data.trades) {
            terminal.currentPrice = data.trades / 10000;
            document.getElementById('priceTicker').textContent = `$${terminal.currentPrice.toFixed(2)}`;

            if (strategyModule.currentStrategy === 'ma_crossover') {
              const smaData = chartModule.priceChart.data.datasets[1].data;
              if (smaData && smaData.length > 0) {
                const latestSMA = smaData[smaData.length - 1];
                strategyModule.checkMACrossoverSignal(terminal.currentPrice, latestSMA);
              }
            }


          } else if (data.orders) {
            console.log("Order update received:", data.orders);
            orderBookModule.fetchOpenOrders();
          } else if (data.orderbook) {
            orderBookDisplayModule.updateOrderBook(data.orderbook);
          }
        };

        websocketModule.ws.onerror = (error) => {
          console.error('WebSocket error:', error);
          utils.displayError("WebSocket connection error.", false);
        };

        websocketModule.ws.onclose = () => {
          console.log('WebSocket closed');
          utils.displayError("WebSocket connection closed. Real-time updates are disabled.", false);
        };
      }
    };

    // ------ Chart Module ------
    const chartModule = {
      priceChart: null,
      initChart: () => {
        const ctx = document.getElementById('priceChart').getContext('2d');
        chartModule.priceChart = new Chart(ctx, {
          type: 'line',
          data: {
            labels: [],
            datasets: [{
              label: 'BTC/USD Price',
              data: [],
              borderColor: '#00ff9d',
              tension: 0.1
            },
            {
              label: 'SMA (20)',
              data: [],
              borderColor: 'yellow',
              tension: 0.1,
              borderDash: [5, 5]
            }]
          },
          options: {
            responsive: true,
            maintainAspectRatio: false,
            scales: {
              y: {
                beginAtZero: false,
                grid: {
                  color: '#333'
                }
              },
              x: {
                grid: {
                  color: '#333'
                }
              }
            }
          }
        }
      });
      setInterval(chartModule.updateChartData, 10000);
    },

    updateChartData: async () => {
      try {
        const response = await axios.get(`${BASE_URL}/md/v2/kline?symbol=${SYMBOL}&resolution=60`);
        const klineData = response.data.data;
        const closes = klineData.c;
        const timestamps = klineData.t;

        const period = 20;
        const smaValues = chartModule.calculateSMA(closes, period);


        const chartData = {
          labels: timestamps.map(t => new Date(t * 1000).toLocaleTimeString()),
          datasets: [{
            label: 'BTC/USD Price',
            data: closes,
            borderColor: '#00ff9d',
            tension: 0.1
          }, {
            label: 'SMA (20)',
            data: smaValues,
            borderColor: 'yellow',
            tension: 0.1,
            borderDash: [5, 5]
          }]
        };

        chartModule.priceChart.data = chartData;
        chartModule.priceChart.update();

      } catch (error) {
        console.error('Chart update error:', error);
        utils.displayError("Failed to update chart data.", false);
      }
    },

    calculateSMA: (data, period) => {
      const sma = [];
      for (let i = period - 1; i < data.length; i++) {
        let sum = 0;
        for (let j = i - period + 1; j <= i; j++) {
          sum += data[j];
        }
        sma.push(sum / period);
      }
      const padding = new Array(period - 1).fill(null);
      return padding.concat(sma);
    }
  };

    // ------ Order Book Display Module ------
    const orderBookDisplayModule = {
      orderBookDiv: document.getElementById('orderBookDisplay'),
      updateOrderBook: (orderBookData) => {
        if (!orderBookData) {
          console.warn("No order book data received.");
          return;
        }

        const bids = orderBookData.bids.slice(0, 10);
        const asks = orderBookData.asks.slice(0, 10);

        let orderBookHTML = '<div>';
        orderBookHTML += '<h6 class="pixel-text text-center">Asks</h6>';
        asks.forEach(ask => {
          orderBookHTML += `<div class="orderbook-item ask"><span>Price: $${(ask[0] / 10000).toFixed(2)}</span><span>Size: ${ask[1]}</span></div>`;
        });
        orderBookHTML += '<hr class="border-success my-2">';
        orderBookHTML += '<h6 class="pixel-text text-center">Bids</h6>';
        bids.reverse().forEach(bid => {
          orderBookHTML += `<div class="orderbook-item bid"><span>Price: $${(bid[0] / 10000).toFixed(2)}</span><span>Size: ${bid[1]}</span></div>`;
        });
        orderBookHTML += '</div>';

        orderBookDisplayModule.orderBookDiv.innerHTML = orderBookHTML;
      }
    };


    // ------ Order Book Module ------
    const orderBookModule = {
      selectedOrderIdToModify: null,
      modifyOrderModal: document.getElementById('modifyOrderModal'),
      modifyQuantityInput: document.getElementById('modifyQuantity'),
      modifyOrderError: document.getElementById('modifyOrderError'),


      fetchOpenOrders: async () => {
        try {
          const response = await apiRequest.signedRequest('/orders/active', 'GET');
          const ordersHtml = orderBookModule.generateOrderList(response.data.rows);
          document.getElementById('openOrders').innerHTML = ordersHtml;
        } catch (error) {
          console.error('Orders fetch error:', error);
          utils.displayError("Failed to fetch open orders.", false);
        }
      },

      fetchOrderHistory: async () => {
        try {
          const response = await apiRequest.signedRequest('/orders/history', 'GET');
          const historyHtml = orderBookModule.generateOrderList(response.data.rows, true);
          document.getElementById('orderHistory').innerHTML = historyHtml;
        } catch (error) {
          console.error('Order history fetch error:', error);
          utils.displayError("Failed to fetch order history.", false);
        }
      },

      generateOrderList: (orders, isHistory = false) => {
        return orders.map(order => {
          const timestamp = isHistory ? ` - ${new Date(order.createdAt).toLocaleString()}` : '';
          const actionButtons = !isHistory ? `
                      <div class="order-actions">
                          <button class="btn btn-danger btn-sm cancel-order-btn" data-order-id="${order.orderID}">Cancel</button>
                          <button class="btn btn-success btn-sm modify-order-btn" data-order-id="${order.orderID}">Modify</button>
                      </div>
                  ` : '';
          return `
                  <div class="order-item mb-2 p-2 bg-secondary" data-order-id="${order.orderID}">
                      <span>${order.side} ${order.orderQty} @ ${order.price || 'Market'} ${timestamp}</span>
                      ${actionButtons}
                  </div>
              `;
        }).join('');
      },

      cancelOrder: async (orderID) => {
        utils.showLoadingOverlay();
        try {
          const cancelData = {
            symbol: SYMBOL,
            orderID: orderID
          };
          await apiRequest.signedRequest('/orders/cancel', 'DELETE', cancelData);
          alert(`Order ${orderID} cancelled successfully!`);
          orderBookModule.fetchOpenOrders();
          orderBookModule.fetchOrderHistory();
        } catch (error) {
          console.error('Cancel order error:', error);
          const errorMessage = error.response?.data?.error?.message || `Failed to cancel order ${orderID}.`;
          utils.displayError(errorMessage, false);
        } finally {
          utils.hideLoadingOverlay();
        }
      },

      openModifyModal: (orderID) => {
        orderBookModule.selectedOrderIdToModify = orderID;
        orderBookModule.modifyOrderModal.style.display = 'block';
        orderBookModule.modifyOrderError.textContent = '';
        orderBookModule.modifyQuantityInput.value = '';
      },


      confirmModifyOrder: async () => {
        const newQuantity = parseFloat(orderBookModule.modifyQuantityInput.value);
        const orderID = orderBookModule.selectedOrderIdToModify;

        if (isNaN(newQuantity) || newQuantity <= 0) {
          orderBookModule.modifyOrderError.textContent = "Please enter a valid quantity to modify to.";
          return;
        }

        utils.showLoadingOverlay();
        try {
          // Assuming Phemex uses /orders/replace endpoint - VERIFY THIS in Phemex API docs
          const modifyData = {
            symbol: SYMBOL,
            orderID: orderID,
            orderQty: newQuantity,
            ordType: 'Market',
            timeInForce: 'ImmediateOrCancel'
          };
          const response = await apiRequest.signedRequest('/orders/replace', 'PUT', modifyData);
          alert(`Order ${orderID} modified successfully!`);
          orderBookModule.fetchOpenOrders();
          orderBookModule.fetchOrderHistory();
          orderBookModule.closeModifyModal();
        } catch (error) {
          console.error('Modify order error:', error);
          const errorMessage = error.response?.data?.error?.message || `Failed to modify order ${orderID}.`;
          orderBookModule.modifyOrderError.textContent = errorMessage;
          utils.displayError(errorMessage, false);
        } finally {
          utils.hideLoadingOverlay();
        }
      },


      closeModifyModal: () => {
        orderBookModule.modifyOrderModal.style.display = 'none';
        orderBookModule.selectedOrderIdToModify = null;
      },


      init: () => {
        const openOrdersDiv = document.getElementById('openOrders');
        openOrdersDiv.addEventListener('click', function(event) {
          if (event.target.classList.contains('cancel-order-btn')) {
            const orderID = event.target.dataset.orderId;
            if (confirm(`Are you sure you want to cancel order ${orderID}?`)) {
              orderBookModule.cancelOrder(orderID);
            } else if (event.target.classList.contains('modify-order-btn')) {
              const orderID = event.target.dataset.orderId;
              orderBookModule.openModifyModal(orderID);
            }
          });

          document.getElementById('closeModifyModal').addEventListener('click', orderBookModule.closeModifyModal);
          document.getElementById('confirmModifyOrder').addEventListener('click', orderBookModule.confirmModifyOrder);

          window.addEventListener('click', (event) => {
            if (event.target == orderBookModule.modifyOrderModal) {
              orderBookModule.closeModifyModal();
            }
          });
      }
    };

    // ------ Positions Module ------
    const positionsModule = {
      fetchPositions: async () => {
        try {
          const response = await apiRequest.signedRequest('/positions', 'GET');
          const positionsHtml = positionsModule.generatePositionsList(response.data.rows);
          document.getElementById('positions').innerHTML = positionsHtml;
        } catch (error) {
          console.error('Positions fetch error:', error);
          utils.displayError("Failed to fetch positions.", false);
        }
      },
      generatePositionsList: (positions) => {
        return positions.map(position => `
                  <div class="position-item mb-2 p-2 bg-secondary" data-symbol="${position.symbol}">
                      ${position.symbol} ${position.side} ${position.qty} @ ${position.entryPrice}
                  </div>
              `).join('');
    }
  };

  // ------ Trading Module ------
  const tradingModule = {
    placeOrder: async (side) => {
      const quantity = 1; // Fixed quantity for strategy for now
      document.getElementById('orderError').textContent = "";
      utils.showLoadingOverlay();

      try {
        const order = {
          symbol: SYMBOL,
          side: side.toUpperCase(),
          orderQty: quantity,
          ordType: 'Market',
          timeInForce: 'ImmediateOrCancel'
        };

        const response = await apiRequest.signedRequest('/orders', 'POST', order);
        alert(`Strategy Order Placed: ${side.toUpperCase()} ${quantity} ${SYMBOL}\nOrder ID: ${response.data.orderID}`);
        orderBookModule.fetchOpenOrders();
        orderBookModule.fetchOrderHistory();
      } catch (error) {
        utils.hideLoadingOverlay();
        console.error('Order error:', error);
        const errorMessage = error.response?.data?.error?.message || "Strategy Order failed. Please check the console for details.";
        utils.displayError(errorMessage);
        return;
      } finally {
        utils.hideLoadingOverlay();
      }
    },

    init: () => {
      document.getElementById('buyButton').addEventListener('click', () => tradingModule.placeOrder('buy')); // Manual Buy button still works
      document.getElementById('sellButton').addEventListener('click', () => tradingModule.placeOrder('sell')); // Manual Sell button still works
    }
  };

  // ------ Strategy Module ------
  const strategyModule = {
    strategySelector: document.getElementById('strategySelector'),
    strategyStatusDisplay: document.getElementById('strategyStatus'),
    strategySignalDisplay: document.getElementById('strategySignal'),
    currentStrategy: 'none',
    currentSignal: 'NONE',
    smaPeriod: 20,


    init: () => {
      strategyModule.strategySelector.addEventListener('change', strategyModule.handleStrategyChange);
      strategyModule.updateStrategyStatus();
    },

    handleStrategyChange: () => {
      strategyModule.currentStrategy = strategyModule.strategySelector.value;
      strategyModule.updateStrategyStatus();
      strategyModule.updateStrategySignal('NONE');

      if (strategyModule.currentStrategy === 'ma_crossover') {
        alert("Moving Average Crossover strategy activated. Automated trading is now LIVE (Example Strategy - Use with caution!)");
      } else if (strategyModule.currentStrategy === 'none') {
        alert("Manual trading mode activated. Automated strategy is OFF.");
      }
    },

    updateStrategyStatus: () => {
      const strategyName = strategyModule.currentStrategy === 'none' ? 'Inactive (Manual Trading)' : strategyModule.strategySelector.options[strategyModule.strategySelector.selectedIndex].text;
      strategyModule.strategyStatusDisplay.textContent = `Strategy: ${strategyName}`;
    },

    updateStrategySignal: (signal) => {
      strategyModule.currentSignal = signal;
      strategyModule.strategySignalDisplay.textContent = `Signal: ${signal}`;
    },


    checkMACrossoverSignal: (currentPriceValue, smaValue) => {
      if (smaValue == null) {
        strategyModule.updateStrategySignal('WAITING_SMA_DATA');
        return 'NONE';
      }

      let signal = 'NONE';
      if (currentPriceValue > smaValue) {
        signal = 'BUY';
      } else if (currentPriceValue < smaValue) {
        signal = 'SELL';
      }

      if (signal !== 'NONE') {
        if (strategyModule.currentSignal !== signal) {
          strategyModule.updateStrategySignal(signal);
          tradingModule.placeOrder(signal.toLowerCase());
        }
      } else {
        strategyModule.updateStrategySignal('NONE');
      }
      return signal;
    },


  };


  // ------ Terminal Initialization Module ------
  const terminal = {
    currentPrice: 0,
    initTerminal: () => {
      utils.showLoadingOverlay(); // Show loading overlay at start
      fetch('/api/keys') // Fetch API keys from Flask server
        .then(response => response.json())
        .then(data => {
          API_KEY = data.apiKey;
          API_SECRET = data.apiSecret;

          websocketModule.initWebSocket();
          chartModule.initChart();
          chartModule.updateChartData();
          orderBookDisplayModule.orderBookDiv = document.getElementById('orderBookDisplay');
          orderBookModule.fetchOpenOrders();
          orderBookModule.fetchOrderHistory();
          positionsModule.fetchPositions();
          strategyModule.init();

          utils.hideLoadingOverlay(); // Hide loading overlay after successful init
        })
        .catch(error => {
          console.error("Failed to fetch API keys:", error);
          utils.displayError("Failed to load API keys from server.", true);
          utils.hideLoadingOverlay(); // Hide loading overlay even on error
        });
    },
    init: () => {
      document.addEventListener('DOMContentLoaded', () => {
        orderBookModule.init();
        tradingModule.init();
        terminal.initTerminal(); // Directly initialize terminal on DOMContentLoaded
      });
    }
  };

  terminal.init();
</script>
</body>

</html>
```

In [ ]:
from google.colab import output
from IPython.display import Javascript, display

# Assuming 'terminal.html' is in the same directory in Colab
# serve_kernel_html was deprecated and should be replaced by serve_kernel_port_as_iframe
from google.colab import output
from IPython.display import Javascript, display

# Assuming 'terminal.html' is in the same directory in Colab
# serve_kernel_html was deprecated and should be replaced by serve_kernel_port_as_iframe
# HTML content is no longer accepted and you need to provide a path instead
# The 'path_to_file' argument should be used instead of 'path'
url = output.serve_kernel_port_as_iframe(port=5001) # Choose a port for the HTML server

display(Javascript(f'window.open("{url}");'))
print(f"Terminal UI is opening in a new tab: {url}")
print(f"Terminal UI is opening in a new tab: {url}")

**To run the terminal:**

1.  **Create Files in Colab:**
    *   Create a file named `api_config.py` and paste the content of the first code block into it.
    *   Create a file named `terminal.html` and paste the content of the third code block into it.
    *   Copy the Flask server code (second code block) and the Colab launcher code (fourth code block) into separate code cells in your Colab notebook.

2.  **Run Cells in Order:**
    *   Execute the code cell containing `api_config.py`.
    *   Execute the code cell containing the Flask server code. **Leave this cell running.**
    *   Execute the code cell containing the Colab launcher code. This will open the terminal in a new tab.

Now you should have the trading terminal running in a new tab in your browser, fetching API keys securely from your `api_config.py` file via the Flask server in your Google Colab environment.

<div class="md-recitation">
  Sources
  <ol>
  <li><a href="https://stackoverflow.blog/2023/04/26/the-worst-coder-in-the-world-tries-an-ai-sidekick/">https://stackoverflow.blog/2023/04/26/the-worst-coder-in-the-world-tries-an-ai-sidekick/</a></li>
  <li><a href="https://github.com/0051Nasrulhakim/skripsi_portal_layanan">https://github.com/0051Nasrulhakim/skripsi_portal_layanan</a></li>
  <li><a href="https://github.com/Gaurav07770/Task-Management-System">https://github.com/Gaurav07770/Task-Management-System</a></li>
  <li><a href="https://github.com/CoffeeHouseGroup/CoffeeHouse">https://github.com/CoffeeHouseGroup/CoffeeHouse</a></li>
  <li><a href="https://github.com/danj98/personal-website">https://github.com/danj98/personal-website</a></li>
  <li><a href="https://stackoverflow.com/questions/76980180/angular-component-not-showing-when-set-to-true">https://stackoverflow.com/questions/76980180/angular-component-not-showing-when-set-to-true</a></li>
  <li><a href="https://github.com/RDxR10/image-gallery">https://github.com/RDxR10/image-gallery</a></li>
  <li><a href="https://github.com/braxtonhall/library-thing">https://github.com/braxtonhall/library-thing</a></li>
  <li><a href="https://github.com/emohawk13/portfoilo">https://github.com/emohawk13/portfoilo</a></li>
  </ol>
</div>